# Intro

**Disadvantages of standard List class**

- The length of dynamic array might be more than the actual number of elements.
- Amortized bounds for operations may not be suitable for real time systems.
- Inserting and removing elements at inferior positions is expensive.

**Linked list is an alternative to an array based sequence**

- Both array based and linked list store elements in an order but using different styles
- List is more centralized representation i.e a large chunk of memory holding references to its elements.
- Linked list relies on a more distributed representation in which a lightweight object i.e `node` is allocated for each element.
  - This node holds the ref to its element and also one or more ref to neighbouring nodes to collectively represent the linear order of the sequence.
  - Elements of a linked list cannot be effectively accessed by a numeric index k. 
  - We can not tell by examining a node whether it is second or tenth node in the list. 
  - However, linked list avoid the 3 disadvantages mentioned above.

# Singly Linked List

- Collection of nodes that form a linear sequence.
- Each node contains a ref to an object that is an element of the sequence as well as a reference to the next node of the list.

![Alt text](pictures/sll_1.png "Singly Linked List")

![Alt text](pictures/sll_2.png "Singly Linked List")

**Traversing Linked List / Link Hopping / Pointer Hopping**

- Starting from the head of the linked list, we can reach the tail using each nodes' `next` member. 
- We can identify the tail with its next reference to None. 

**Linked list is a collaboration of many objects**

- list itself
  - head
  - tail
  - size
- node
  - element
  - next

Minimally, the linked list should have a reference to the head of the list. Without an explicit ref to head, there is no way to locate that node or indirectly others.

![Alt text](pictures/sll_3.png "Singly Linked List")


### Inserting an element at head

```
add_first(L,e):

  new = Node(e)
  new.next = L.head
  L.head = new
  L.size += 1 
```

### Inserting an element at tail

```
add_last(L,e):
  new = Node(e)
  new.next = None
  L.tail.next = new
  L.tail = new
  L.size +=1
```

### Removing head

```
remove_first(L):
  if L.head is None then
    indicate an error - List is empty

  L.head = L.head.next
  L.size -= 1
```

### Removing tail

We cannot easily delete the last node, since we must be able to access the node before the last node. Only way is to access the(tail-1) node is to traverse the list which would be inefficient. If we want to support that operation, we would have to use doubly linked list.

## Implementing a stack using singly linked list

We model the the top of the stack at the head of the list since we can efficiently insert and delete elements in constant time only at the head. 

All of its methods will complete in worst case constant time in contrast to amortized bounds for the ArrayStack

In [7]:
class Empty(Exception):
    """Error attempting to access an element from an empty container."""

    pass


class LinkedStack:
    class _Node:
        __slots__ = ("_element", "_next")

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._size = 0

    def push(self, element):
        node = self._Node(element, self._head)
        self._head = node
        self._size += 1

    def pop(self):
        if self.is_empty():
            return Empty
        element = self._head._element
        self._head = self._head._next
        self._size -= 1

        return element

    def top(self):
        return self._head._element

    def is_empty(self):
        return self._size == 0

    def __len__(self):
        return self._size


s = LinkedStack()

for i in range(10):
    s.push(i)

for i in range(5):
    print(s.pop())

print(s.top())
print(len(s))

9
8
7
6
5
4
5


## Implementing a queue using single linked list

- With worst-case O(1) time for all operations. 
- Space usage is linear in the current number of elements.
- We will need both _head and _tail ref since we need to operate at both ends
- first element of queue will be modeled as the head of the linked list. And last element of queue will be modeled as tail of linked list. 
  - Because we can not remove the tail of the linked list because of the lack of previous reference.

In [18]:
class LinkedQueue:
    class _Node:
        __slots__ = ("_element", "_next")

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def enqueue(self, element):
        new = self._Node(element, None)
        if not self.is_empty():
            self._tail._next = new
        else:
            self._head = new
        self._tail = new
        self._size += 1

    def dequeue(self):
        if self.is_empty():
            raise Empty

        element = self._head._element
        self._head = self._head._next
        self._size -= 1

        if self.is_empty():
            self._tail = None
            self._head = None

        return element

    def first(self):
        if self.is_empty():
            raise Empty
        element = self._head._element
        return element

    def is_empty(self):
        return self._size == 0

    def __len__(self):
        return self._size


q = LinkedQueue()

for i in range(10):
    q.enqueue(i)

for i in range(10):
    print(q.dequeue())

print(len(q))

q.enqueue(11)
q.enqueue(12)

print(q.dequeue())
print(q.dequeue())

0
1
2
3
4
5
6
7
8
9
0
11
12


# Circular Linked List

- Earlier while implementing `ArrayQueue`, we demonstrated use of `circularArray`. 
- This was artificial as in there was nothing about the representation of the array that was circular in structure.
- In case of linked lists, there is more tangible notion of this circularity. 
- This provides a more general model for data sets that are cyclic i.e which do not have any particular notion of beginning and end. 
- Even though a circular linked list does not have a beginning or end, we must maintain a ref to a particular node in order to make use of the list. We use the identifier `current` to describe such a node. 

![Alt text](pictures/cll_1.png "Circular Linked List")

## Round-Robin Schedulers 

- This is one of the use case example of circular linked list
- Round robin scheduler iterates through a collection of elements in a circular fashion and "services" that element by performaing an action on it.
- Such a scheduler is used for example to fairly allocate a resource. 
- Round robin scheduling is often used to allocate slices of CPU time to various applications running concurrently.

- A round robin scheduler could be implemented with the general queue ADT.
  - e = q.dequeue()
  - service element e
  - q.enqueue(e)

![Alt text](pictures/rrs_1.png "Round Robin scheduler")

- Using a simple LinkedQueue for this, there will be unnecessary effort in the combination of dequeue followed soon after by an enqueue of the same element
- If using a circular list, this operation of transfer of head to tail can be done more efficiently.

  

In [32]:
class CircularQueue:
    class _Node:
        __slots__ = ("_element", "_next")

        def __init__(self, element, next):
            self._element = element
            self._next = next

    def __init__(self):
        self._tail = None
        self._size = 0

    def enqueue(self, element):
        if not self.is_empty():
            new = self._Node(element, self._tail._next)
            self._tail._next = new
        else:
            new = self._Node(element, None)
            new._next = new
        self._tail = new
        self._size += 1

    def rotate(self):
        if self._size > 1:
            self._tail = self._tail._next

    def dequeue(self):
        if self.is_empty():
            raise Empty

        element = self._tail._next._element
        self._tail._next = self._tail._next._next
        self._size -= 1

        if self.is_empty():
            self._tail = None

        return element

    def first(self):
        if self.is_empty():
            raise Empty
        element = self._tail._next._element
        return element

    def is_empty(self):
        return self._size == 0

    def __len__(self):
        return self._size


q = CircularQueue()

for i in range(10):
    q.enqueue(i)

for i in range(5):
    print(q.dequeue())

print(q.first())

q.rotate()

print(q.first())

for i in range(5):
    print(q.dequeue())

0
1
2
3
4
5
6
6
7
8
9
5


# Doubly Linked Lists

- These lists allow a greater variety of O(1) time update operations including insertions and deletions at arbitrary positions.

**Header and Trailer Sentinels**

- For some special cases, it helps to add special nodes at both ends of the list : header node and trailer node. 
- These are dummy nodes and are known as sentinels or guards, and they do not store elements of the primary sequence.
- Advantage of having sentinels is to treat all insertions in an unified manner because
  - a new node will always be inserted b/w two nodes.
  - every node to be deleted will have neighbours. 

![Alt text](pictures/dll_1.png "Round Robin scheduler")

- Linked lists can support general insertions and deletions in O(1) worst case time but only if the location of an operation can be succintly identified.
- With array based, an int index was a convenient means of describing a position within a sequence. However, it would require to traverse the linked list for the same. 



In [35]:
class _DoubleLinkedList:
    class _Node:
        __slots__ = ("_next", "_prev", "_element")

        def __init__(self, element, next, prev):
            self._element = element
            self._next = next
            self._prev = prev

    def __init__(self) -> None:
        self._size = 0
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer
        self._trailer._prev = self._header

    def is_empty(self):
        return self._size == 0

    def __len__(self):
        return self._size

    def _insert_between(self, element, left, right):
        new = self._Node(element, right, left)
        left._next = new
        right._prev = new
        self._size += 1
        return new

    def _delete_node(self, node):
        if self.is_empty():
            raise Empty
        element = node._element

        left = node._prev
        right = node._next

        left._next = right
        right._prev = left

        # deprecating node
        node._prev = node._next = node._element = None

        return element

# Implementing Deck with doubly linked list

- Array based implementation : O(1) amortized time due to occasional need to resize the array
- Doubly linked list based implementation : O(1) worst case time

In [42]:
class LinkedDeck(_DoubleLinkedList):
    def __init__(self):
        super().__init__()

    def first(self):
        if self.is_empty():
            raise Empty

        return self._header._next._element

    def last(self):
        if self.is_empty():
            raise Empty

        return self._trailer._prev._element

    def pop(self):
        if self.is_empty():
            raise Empty

        element = self._delete_node(self._trailer._prev)

        return element

    def pop_left(self):
        if self.is_empty():
            raise Empty

        element = self._delete_node(self._header._next)

        return element

    def append(self, element):
        self._insert_between(element, self._trailer._prev, self._trailer)

    def append_left(self, element):
        self._insert_between(element, self._header, self._header._next)


deck = LinkedDeck()

for i in range(5):
    deck.append(i)

print(deck.pop())
print(deck.pop_left())

for i in range(10, 15):
    deck.append_left(i)

print(deck.pop_left())

4
0
14


# The Positional List ADT

- The ADTs stack, queue and linked list only allow operations at the ends of the sequence. 
- We need to design an ADT which allows referring elements anywhere in the sequence and to perform arbitrary insertions and deletions. 
- Numeric indices not a good choice for describing position in a linked list since finding an element would require traversing the list. 
- Indices not a good abstraction - because index of an entry changes over time due to insertion or deletions. 
- As another example - a text document having cursor to describe a position without use of explicit index.

**A node reference as position??**

- It is possible to do O(1) worst time arbitrary insertions and deletions as long as we are given a ref to a relevant node of the list. 
- However using node ref as positions violates the OOPs encapsulation and abstraction. 
- Hence, a need of position ADT. 
- Position p is unaffected by changes elsewhere in a list.


**Behaviours**

- Position p is a simple obj supporting only 1 method
  - p.element() : returns the element stored at position p. 
- L.first(): returns position of first element. None if L is empty.
- L.last(): returns position of last element. None if L is empty. 
- L.before(p): returns position of L immediately before p. None if p is the first element.
- L.after(p): returns position of L immediately after p. None if p is the last element. 
- L.is_empty()
- len(L)
- iter(L) : returns iterator for elements of list

- L.add_first(e): inserts a new element in front of L. Returns position of new element.
- L.add_last(e): inserts a new clement at back of L. Returns position of new element.
- L.add_before(p): inserts a new element just before position p. Returns position of new element. 
- L.add_after(p): inserts a new element just after position p. Returns position of new element. 
- L.replace(p,e): replaces element at position p with element e returning element formerly at position p. 
- L.delete(p): removes and returns element at position p. Also invalidates the position

**Implementation**

- Using doubly linked list
- We could end up creating redundant position instances that reference the same underlying node, for this we will need `__eq__` and `__ne__` special methods.

In [ ]:
class PositionalList(_DoubleLinkedList):
    class _Position:
        __slots__ = ("_container", "_node")

        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        def __eq__(self, other):
            return type(self) is type(other) and self._node is other._node

        def __ne__(self, other):
            return not (self == other)

    def __init__(self):
        super().__init__()

    def _validate(self, p):
        if not isinstance(p, self._Position):
            raise TypeError("p must be proper Position type")
        if p.container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._next is None:
            return ValueError("p is no longer valid")

        return p._node

    def _make_position(self, node):
        if node is self._header or node is self._trailer:
            return None
        else:
            self._Position(self, node)

    def first(self):
        return self._make_position(self._header._next)

    def last(self):
        return self._make_position(self._trailer._prev)

    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        p = self.first()
        while p is not None:
            yield p.element()
            p = self.after(p)

    def _insert_between(self, element, left, right):
        node = super()._insert_between(element, left, right)
        return self._make_position(node)

    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        right = self._validate(p)

        return self._insert_between(e, right._prev, right)

    def add_after(self, p, e):
        left = self._validate(p)

        return self._insert_between(e, left, left._next)

    def delete(self, p):
        return self._delete_node(self._validate(p))

    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value

## Insertion sort of positional list

- marker: represents rightmost position of the currently sorted portion of list
- pivot: during each pass, we consider position just after marker as pivot and try to move pivot to its correct location in the sorted portion
- walk : we mantain this to move leftward from the marker as long as there is a preceding element with value larger than the pivot.

![Alt text](pictures/is_pl.png "Singly Linked List")


In [ ]:
def insertion_sort(positional_list):
    marker = positional_list.first()

    while marker != positional_list.last():
        pivot = positional_list.after(marker)
        value = pivot.element()

        if value > marker.element():
            marker = pivot
        else:
            walk = positional_list.before(marker)
            while walk is not None and walk.element() > pivot.element():
                walk = positional_list.before(walk)

            positional_list.delete(pivot)
            positional_list.add_before(walk, value)

        marker = positional_list.after(marker)

# Favorites list

- example of a web browser that keeps track of user's most frequently visited urls. 
- We model this with a favorites list ADT that supports the following methods
  - access(e): access element e, incrementing its access count and adding it to the fav list if not already present
  - remove(e): removes element e from the fav list if present
  - top(k): return iterator of the k most accessed elements.
- We access or remove an element by searching the list from the most frequently accessed to the least frequently accessed

In [ ]:
class FavoritesList:
    class _Item:
        __slots__ = ("_value", "_count")

        def __init_(self, value, count):
            self._value = value
            self._count = count

    def __init__(self):
        self._data = PositionalList()

    def _find_position(self, element):
        walk = self._data.first()
        while walk is not None and walk.element()._value != element:
            walk = self._data.after(walk)

        return walk

    def __len__(self):
        return len(self._data)

    def is_empty(self):
        return len(self._data) == 0

    def top(self, k):
        if not 1 <= k <= len(self):
            raise ValueError("Illegal value for k")

        walk = self._data.first()

        for _ in range(k):
            item = walk.element()
            yield item._value
            walk = self._data.after(walk)

    def remove(self, element):
        p = self._find_position(element)

        if p is not None:
            self._data.delete(p)

    def _move_up(self, p):
        # consider moving
        if p != self._data.first():
            walk = self._data.before(p)
            count = p.element()._count

            # must shift the item to left side
            if walk.element()._count < count:
                while walk is not None and walk.element()._count < count:
                    walk = self._data.before(walk)

                # delete/reinsert
                self._data.add_before(walk, self._data.delete(p))

    def access(self, element):
        p = self._find_position(element)

        if p is None:
            p = self._data.add_last(self._Item(element))

        p.element()._count += 1

        self._move_up(p)

# Move-to-Front Heuristic in fav list

- The previous fav list implementation performs the `access(e)` method in time proportional to index of element e in the list. So, if e is kth element, accessing it takes O(k) time.
- In many use cases like web pages visited by user, once an element is accessed, it is more likely to be accessed in the near future. Such scenarios are said to possess `locality of reference` in computing. 
- A heuristic to take advantage of locality of reference in an access sequence in the *move-to-front heuristic*. 
- Each time we access an element, we move it all the way to the front of the list.
- Trade offs
  - The top method will now have to search for the k most accessed elements which means it is going to take O(kn) time where k is a constant. But if k is proportional to n, then top runs in `O(n**2)` for e.g if we want top 25% list.

In [ ]:
class FavoritesListMTF(FavoritesList):
    def _move_up(self, p):
        if p != self._data.first():
            self._data.add_first(self._data.delete(p))

    def top(self, k):
        if not 1 <= k <= len(self):
            raise ValueError("Illegal value for k")

        temp = PositionalList()

        for item in self._data:
            temp.add_last(item)

        for _ in range(k):
            high_position = temp.first()
            walk = temp.after(high_position)

            while walk is not None:
                if walk.element()._count > high_position.element()._count:
                    high_position = walk
                walk = temp.after(walk)

            yield high_position.element()._value

            temp.delete(high_position)

# Link-based vs Array-based sequences

**Advantages of array based sequences**

- Arrays provide O(1) time access to an element based on integer index. While the same for linked list would be O(n). 
- Operations with equivalent asymptotic bounds typically run a constant factor more efficiently with an array based structure vs a linked structure. 
- Array based sequences usually use proportionally less memory than linked structures.

**Advantages of link based sequences**

- Link based structures provide worst case time bound rather than amortized based. This is critica for real time systems.
- Link based structures support O(1) insertions and deletions at arbitrary positions given a position that represents the location in the list. 
